## Predictions on intermediate years using best model

In [1]:
import pickle

In [2]:
emp_femtss = pickle.load(open('xgBoost_models/emp_femaletop_score_specs.pkl','rb'))
print(emp_femtss['xgBoost']['val_scores'])
print('-------------')
print(emp_femtss['xgBoost']['train_scores'])

[0.7401195411888216, 0.7248345257490144, 0.7105458596357251, 0.7043082789836379, 0.6973348478544594]
-------------
[1.0, 1.0, 1.0, 1.0, 1.0]


## Tesing on MSL-XGBoost 

In [3]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import numpy as np
def apply_smote(data, feature_cols, target):
    '''
    Input:
    data - the original dataframe
    feature_cols - the feature columns (list of columns)
    target - the target column (string value)
    '''
    sm = SMOTE(random_state=42)
    features, targets = sm.fit_resample(data[feature_cols],data[target])
    feature_df = pd.DataFrame(features, columns=feature_cols)
    target_df = pd.DataFrame(targets, columns=[target])
    output = pd.concat([feature_df, target_df], axis=1)
    # Shuffling dataset
    output = output.sample(frac=1).reset_index(drop=True)
    return output
filepath = '2011_districts_10_bins_v2.csv'
data = pd.read_csv(filepath)
feature_cols = data.columns[1:-10].tolist()

In [4]:
data.columns

Index(['censuscode', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       'MSW+AF8-2011', 'BF+AF8-2011', 'MSL+AF8-2011', 'FC+AF8-2011',
       'CHH+AF8-2011', 'EMP+AF8-2011', 'ASSET+AF8-2011', 'EMP_FEMALE+AF8-2011',
       'LIT+AF8-2011', 'EMP_AG_NONAG+AF8-2011'],
      dtype='object', length=131)

In [5]:
data['EMP_FEMALE+AF8-2011'] = data['EMP_FEMALE+AF8-2011'] -1
y_col_EMP_FEMALE = 'EMP_FEMALE+AF8-2011'
output = apply_smote(data, feature_cols, y_col_EMP_FEMALE)
print(output['EMP_FEMALE+AF8-2011'].value_counts())
X = output[feature_cols].values
y = output[y_col_EMP_FEMALE].values

print(X.shape)
print(y.shape)

2    243
1    243
0    243
Name: EMP_FEMALE+AF8-2011, dtype: int64
(729, 120)
(729,)


In [6]:
emp_femtss['xgBoost']['specs'][0]

{'n_estimators': 1000,
 'max_depth': 10,
 'learning_rate': 0.05,
 'verbosity': 1,
 'objective': 'binary:logistic',
 'booster': 'dart',
 'n_jobs': -1,
 'gamma': 0,
 'min_child_weight': 0,
 'max_delta_step': 0,
 'subsample': 1,
 'colsample_bytree': 1,
 'colsample_bylevel': 1,
 'colsample_bynode': 0.7,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'base_score': 0.5,
 'random_state': 0,
 'importance_type': 'weight',
 'n_randomized_search': 200,
 'run_type': 'train-test',
 'trail': 0,
 'trail_type': 'sales',
 'train_ratio': 0.9,
 'val_train_loss': 'f1_weighted',
 'top_n_models_drill': 5,
 'top_n_models_pred': 5,
 'n_drill_search': [50],
 'test_ratio': 0.0,
 'y_col': 'EMP_FEMALE+AF8-2011',
 'cols_to_drop': [],
 'ensemble': False,
 'n_cols_dropped': 0,
 'current_date_col': 'random_drop',
 'no_steps_ahead': 0,
 'time_cols': [],
 'cross_validation': ['KFold'],
 'kFold_splits': 5,
 'pFold_splits': 5,
 'feature_cols': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '

In [7]:
n_estimators = emp_femtss['xgBoost']['specs'][0]['n_estimators']
max_depth = emp_femtss['xgBoost']['specs'][0]['max_depth']
learning_rate = emp_femtss['xgBoost']['specs'][0]['learning_rate']
objective = emp_femtss['xgBoost']['specs'][0]['objective']
booster = emp_femtss['xgBoost']['specs'][0]['booster']
gamma = emp_femtss['xgBoost']['specs'][0]['gamma']
min_child_weight = emp_femtss['xgBoost']['specs'][0]['min_child_weight']
max_delta_step = emp_femtss['xgBoost']['specs'][0]['max_delta_step']
subsample = emp_femtss['xgBoost']['specs'][0]['subsample']
colsample_bytree = emp_femtss['xgBoost']['specs'][0]['colsample_bytree']
colsample_bylevel = emp_femtss['xgBoost']['specs'][0]['colsample_bylevel']
colsample_bynode = emp_femtss['xgBoost']['specs'][0]['colsample_bynode']
reg_alpha = emp_femtss['xgBoost']['specs'][0]['reg_alpha']
reg_lambda = emp_femtss['xgBoost']['specs'][0]['reg_lambda']
scale_pos_weight = emp_femtss['xgBoost']['specs'][0]['scale_pos_weight']
base_score = emp_femtss['xgBoost']['specs'][0]['base_score']

In [8]:
xgbc = XGBClassifier(n_estimators=n_estimators, 
                     max_depth=max_depth, learning_rate=learning_rate, 
                     objective=objective, booster=booster,n_jobs=-1, 
                     gamma=gamma, min_child_weight=min_child_weight, 
                     max_delta_step=max_delta_step, subsample=subsample, 
                     colsample_bytree=colsample_bytree, 
                     colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
                     reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
                     scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

In [9]:
n_splits = 5
cv = KFold(n_splits=n_splits, shuffle=True)
val_score = []
train_score = []
for train_index, test_index in cv.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    best_xgb = xgbc.fit(X_train, y_train)
    
    predictions = best_xgb.predict(X_test)
    predictions_train = best_xgb.predict(X_train)
    f1_weight = f1_score(y_test, predictions, average='weighted')
    f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
    
    val_score.append(f1_weight)
    train_score.append(f1_weight_train)

In [10]:
val_score = np.array(val_score)
print(val_score.mean())
train_score = np.array(train_score)
print(train_score.mean())

0.7407195661869101
1.0


In [11]:
best_xgb

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bynode=0.7, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=0, missing=None, n_estimators=1000, n_jobs=-1,
       nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [12]:
pickle.dump(best_xgb, open('best_xgb_emp_fem.pkl','wb'))

## Generating predictions

In [13]:
best_xgb = pickle.load(open('best_xgb_emp_fem.pkl','rb'))

In [14]:
folder = r"C:\Users\AJain7\OneDrive - Stryker\Personal\Projects\Satellite Project\5. District_level\Change Classifier\Archive_Anuj-Anupam\quantile_csv_past"

In [15]:
output_list = []
for files in os.listdir(folder):
    filepath = os.path.join(folder, files)
    year = files[:4]
    df = pd.read_csv(filepath)
    feature_cols = df.columns[1:]
    df = df[feature_cols]
#     print(df)
    for index, row in df.iterrows():
        X_test = row[1:]
        prediction = best_xgb.predict(X_test)
        output_list.append([year,prediction[0], row[0]])

In [16]:
output_list[56]

['2001', 2, 57]

In [17]:
output = pd.DataFrame(data=output_list,columns=['year', 'predictions', 'district_code'])

In [18]:
output.to_csv('predictions_past_data_emp_fem.csv', index=False)

In [19]:
# filee = r"C:\Users\AJain7\OneDrive - Stryker\Personal\Projects\Satellite Project\5. District_level\Change Classifier\Archive_Anuj-Anupam\quantile_csv\2001_districts_quant.csv"

# ff = pd.read_csv(filee)

# feature_cols = ff.columns[1:]

# ff = ff[feature_cols]

# output = pd.DataFrame(data=output_list,columns=['year', 'district_code', 'predictions'])

# output_list = []
# for index, row in ff.iterrows():
#     X_test = row[1:]
#     prediction = best_xgb.predict(X_test)
# #     print(prediction[0], row[0], '2011')
#     output_list.append(['2011',prediction[0], row[0]])